# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = 'city.csv'
city_df = pd.read_csv(file)
city_df

,Cities,Latitude,Longitude,Temperature(F),Humidity(%),Cloudiness(%),Wind Speed(MPH),Country,Datetime
0,San Jorge,-31.90,-61.86,64.00,28,0,4.00,AR,1590953101
1,Turukhansk,65.82,87.98,45.55,61,100,15.50,RU,1590953101
2,Busselton,-33.65,115.33,55.99,74,100,3.29,AU,1590953101
3,Kadoma,34.73,135.58,66.99,75,44,3.53,JP,1590953101
4,Lebu,-37.62,-73.65,53.96,64,47,17.54,CL,1590953101
...,...,...,...,...,...,...,...,...,...
557,Margate,51.38,1.39,60.01,72,30,4.00,GB,1590953188
558,Nanortalik,60.14,-45.24,43.48,71,100,2.62,GL,1590953188
559,Opuwo,-18.06,13.84,65.66,27,0,9.48,NaN,1590953188
560,Mezen',65.85,44.24,37.13,80,100,6.42,RU,1590953188


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
coordinates = []
coordinates = list(zip(city_df['Latitude'], city_df['Longitude']))
humidity = city_df['Humidity(%)'].astype(float)

In [4]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
max_temp_df = city_df.loc[(city_df['Temperature(F)'] > 70) & (city_df['Temperature(F)'] < 80)]
wind_df = max_temp_df.loc[max_temp_df['Wind Speed(MPH)'] < 10]
hotel_df = wind_df.loc[wind_df['Cloudiness(%)'] == 0]
hotel_df

,Cities,Latitude,Longitude,Temperature(F),Humidity(%),Cloudiness(%),Wind Speed(MPH),Country,Datetime
97,Quang Ngai,15.12,108.80,78.17,84,0,4.23,VN,1590953117
145,Khāsh,28.22,61.22,74.62,14,0,4.29,IR,1590953124
149,Yima,34.74,111.88,78.01,99,0,7.85,CN,1590953124
443,Kiryat Gat,31.61,34.76,71.01,75,0,7.63,IL,1590952922
473,Mastung,29.80,66.84,72.48,36,0,5.91,PK,1590953173
516,Ipameri,-17.72,-48.16,74.44,51,0,2.77,BR,1590953180


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name']=""
hotel_df['Address']=""
hotel_df['Hotel Lat']=""
hotel_df['Hotel Lon']=""

C:\Users\oberm\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\oberm\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\oberm\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "lodging",
    "key": g_key,
    'radius': '5000'
}

for index, row in hotel_df.iterrows():

    # get restaurant type from df
    hotel_lat = row['Latitude'] 
    hotel_lon = row['Longitude']
    params['location'] = (f'{hotel_lat},{hotel_lon}')

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['Cities']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Address'] = results[0]['vicinity']
        hotel_df.loc[index, 'Hotel Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel Lon'] = results[0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 97: Quang Ngai.
Closest hotel is Central Hotel.
------------
Retrieving Results for Index 145: Khāsh.


C:\Users\oberm\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is دفتر پیشخوان دولت جمال زهی (خالد).
------------
Retrieving Results for Index 149: Yima.
Closest hotel is Yima Hotel.
------------
Retrieving Results for Index 443: Kiryat Gat.
Closest hotel is רגע בנחת.
------------
Retrieving Results for Index 473: Mastung.
Closest hotel is Syed House.
------------
Retrieving Results for Index 516: Ipameri.
Closest hotel is Real Hotel.
------------


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lon"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=[f'{i}' for i in hotel_info])
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…